# APEx STAC catalog editor guide

This notebook demonstrates how to add metadata to an APEx STAC catalog.

The STAC API is a standard, so other STAC documentation is also relevant.

## Creating STAC metadata from scratch

In this first part, we show how to start from zero with a very basic example.

In most real-world cases, it is not needed nor recommended to author STAC metadata from scratch directly. Instead, various tools and platforms will generate STAC metadata for you.
The STAC metadata generated here is entirely fictional, and also very much useless. It is not an example of how to create high quality metadata that complies with FAIR principles.

This example mostly serves as a demonstration of the API, which is very simply to use. Most of the more complex examples and tools will avoid this type of direct interaction.


In [11]:

import requests
from owslib.ogcapi.records import Records
from owslib.util import Authentication

In [23]:
from requests_auth import OAuth2ResourceOwnerPasswordCredentials

auth = OAuth2ResourceOwnerPasswordCredentials(
        token_url="https://sso-int.terrascope.be/auth/realms/terrascope/protocol/openid-connect/token",
        username="xxx",
        password="yyy",
        client_id="terracatalogueclient",
    )

In [24]:

r = Records("https://stac-openeo-dev.vgt.vito.be/",auth=Authentication(auth_delegate=auth))
r

In [19]:
from datetime import datetime
import pystac

spatial_extent = pystac.SpatialExtent(bboxes=[[4,51,5,52]])
collection_interval = sorted([datetime(2024,10,1), datetime(2024,11,1)])
temporal_extent = pystac.TemporalExtent(intervals=[collection_interval])

collection_extent = pystac.Extent(spatial=spatial_extent, temporal=temporal_extent)
collection = pystac.Collection(id='apex_unique_collection_id',
                               description='My project generated this fantastic collection.',
                               extent=collection_extent,
                               license='CC-BY-SA-4.0')
collection

<Collection id=apex_unique_collection_id>

In [34]:
coll_dict = collection.to_dict()

default_auth = {
    "_auth": {
        "read": ["anonymous"],
        "write": ["stac-openeo-admin", "stac-openeo-editor"]
    }
}

coll_dict.update(default_auth)

response = requests.post("https://stac-openeo-dev.vgt.vito.be/collections", auth=auth,json=coll_dict)
response

<Response [201]>

In [37]:
import shapely

item_bbox = [4, 51, 4.5, 51.5]
collection_item = pystac.Item(id='my_unique_item',
                              geometry=shapely.geometry.mapping(shapely.box(*item_bbox)),
                              bbox=item_bbox,
                              datetime=datetime(2024,10,2),
                              collection=collection.id,
                              properties={})

collection_item.common_metadata.gsd = 10
collection_item.common_metadata.platform = 'Sentinel-2'
collection_item.common_metadata.instruments = ['MSI']

asset = pystac.Asset(href="https://myhost.com/my_cog.tif", 
                      media_type=pystac.MediaType.GEOTIFF)
collection_item.add_asset("the_asset_key", asset)
collection_item

<Item id=my_unique_item>

In [38]:
r.collection_item_create(collection.id, collection_item.to_dict())

True

## Cleaning up

When the demo is done, we can simply clean up by deleting our (useless) test collection.

In [26]:
requests.delete("https://stac-openeo-dev.vgt.vito.be/collections/" + collection.id, auth=auth)

<Response [204]>